In [1]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import timm
import os
from tqdm import tqdm
from natsort import natsorted
from PIL import Image
from glob import glob
import numpy as np

In [2]:
model_name = 'swin_small_patch4_window7_224'
model = timm.create_model(model_name, pretrained=True, num_classes=1000, in_chans=3)

/home/brainimage/anaconda3/envs/brainimage/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525552843/work/aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [3]:
quantized_model = torch.quantization.quantize_dynamic(model, {torch.nn.Linear}, dtype=torch.qint8)

In [4]:
device = 'cpu'

In [5]:
# define the transformation to be applied to the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # resize the image to match the model input size
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

images = natsorted(glob('/home/brainimage/filters/val/*'))
labels = open('./labels.txt', 'r').readlines()

print(images[:100])

for i in tqdm(range(100)):
    images[i] = Image.open(images[i])
    images[i] = images[i].convert('RGB')
    images[i] = transform(images[i])
    labels[i] = int(labels[i].strip().split(' ')[1])

images = images[:100]
images = torch.stack(images)
labels = labels[:100]

images = torch.tensor(images)
labels = torch.tensor(labels)


test_dataset = torch.utils.data.TensorDataset(images, labels)

# create the test dataset
test_dataset = torch.utils.data.TensorDataset(images, labels)

## perfrom slicing of the dataset
test_dataset = torch.utils.data.Subset(test_dataset, range(100))

# create the test loader
test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=100,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)


['/home/brainimage/filters/val/ILSVRC2012_val_00000001.JPEG', '/home/brainimage/filters/val/ILSVRC2012_val_00000002.JPEG', '/home/brainimage/filters/val/ILSVRC2012_val_00000003.JPEG', '/home/brainimage/filters/val/ILSVRC2012_val_00000004.JPEG', '/home/brainimage/filters/val/ILSVRC2012_val_00000005.JPEG', '/home/brainimage/filters/val/ILSVRC2012_val_00000006.JPEG', '/home/brainimage/filters/val/ILSVRC2012_val_00000007.JPEG', '/home/brainimage/filters/val/ILSVRC2012_val_00000008.JPEG', '/home/brainimage/filters/val/ILSVRC2012_val_00000009.JPEG', '/home/brainimage/filters/val/ILSVRC2012_val_00000010.JPEG', '/home/brainimage/filters/val/ILSVRC2012_val_00000011.JPEG', '/home/brainimage/filters/val/ILSVRC2012_val_00000012.JPEG', '/home/brainimage/filters/val/ILSVRC2012_val_00000013.JPEG', '/home/brainimage/filters/val/ILSVRC2012_val_00000014.JPEG', '/home/brainimage/filters/val/ILSVRC2012_val_00000015.JPEG', '/home/brainimage/filters/val/ILSVRC2012_val_00000016.JPEG', '/home/brainimage/filte

100%|██████████| 100/100 [00:15<00:00,  6.29it/s]
/tmp/ipykernel_336109/404962316.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  images = torch.tensor(images)


In [6]:
print(images.shape)

torch.Size([100, 3, 224, 224])


In [7]:
def evaluate(model):
    model.eval()

    with torch.no_grad():
        correct = 0
        total = 0

        for images, labels in tqdm(test_loader):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            print(predicted)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('Accuracy of the network on the 100 test images: %d %%' % (100 * correct / total))

evaluate(quantized_model)


100%|██████████| 1/1 [00:24<00:00, 24.11s/it]

tensor([ 65, 795, 230, 809, 520,  60, 334, 415, 674, 332, 109, 286, 370, 757,
        595, 147, 327,  23, 478, 517, 334, 208, 948, 727,  23, 846, 270, 166,
         55, 538, 324, 573, 360, 981, 586, 887,  26, 398, 777,  74, 431, 756,
        129, 198, 256, 725, 565, 166, 717, 467,  92,  29, 844, 591, 359, 468,
        154, 994, 872, 588, 735, 197, 107,  46, 842, 390, 101, 887, 870, 911,
          4, 149,  21, 476,  80, 424, 159, 275, 175, 461, 970, 160, 788,  58,
        479, 498, 368,  28, 487,  50, 270, 383, 366, 780, 373, 705, 330, 142,
        949, 349])
Accuracy of the network on the 100 test images: 81 %


In [8]:
torch.save(quantized_model.state_dict(), 'quantized_model.pth')
torch.save(model.state_dict(), 'model.pth')

In [9]:
# Evaluate the original model
evaluate(model)

# Get the size of the original model
orig_size = os.path.getsize('model.pth')
print(f"Original model size: {orig_size} bytes")

# Get the size of the quantized model
quantized_size = os.path.getsize('quantized_model.pth')
print(f"Quantized model size: {quantized_size} bytes")


100%|██████████| 1/1 [00:27<00:00, 27.71s/it]

tensor([ 65, 795, 230, 809, 520,  60, 334, 415, 674, 332, 109, 286, 370, 757,
        595, 147, 327,  23, 478, 517, 334, 208, 948, 727,  23, 846, 270, 166,
         55, 538, 324, 573, 360, 981, 586, 887,  26, 398, 777,  74, 431, 756,
        129, 198, 256, 725, 565, 166, 717, 467,  92,  29, 844, 591, 359, 468,
        154, 994, 872, 588, 735, 197, 107,  46, 842, 390, 101, 887, 870, 911,
          4, 149,  21, 476,  80, 424, 159, 275, 175, 461, 970, 160, 788,  58,
        479, 498, 368,  28, 487,  50, 270, 383, 366, 780, 373, 705, 330, 142,
        949, 349])
Accuracy of the network on the 100 test images: 81 %
Original model size: 200110797 bytes
Quantized model size: 51924361 bytes


## As an advanced task

In [10]:
def get_model_size(model):
    torch.save(model.state_dict(), "temp.p")
    size = os.path.getsize("temp.p")
    os.remove("temp.p")
    return size

In [11]:
def quantize_model(model):
    # Set quantization parameters for activations and weights
    activation_observer = torch.quantization.observer.HistogramObserver.with_args(dtype=torch.quint4x2, qscheme=torch.per_tensor_affine)
    quantization_config = torch.quantization.QConfig(activation=activation_observer, weight=torch.quantization.default_weight_observer)
    # Prepare the model for quantization
    model.qconfig = quantization_config
    torch.backends.quantized.engine = 'qnnpack'
    model_prepared = torch.quantization.prepare(model, inplace=True)
    # Calibrate the model to find appropriate quantization parameters
    # with torch.no_grad():
    #     for images, labels in test_loader:
    #         model_prepared(images)
    evaluate(model_prepared)
    # Convert the model to a quantized model
    model_quantized = torch.quantization.convert(model_prepared, inplace=True)
    # Print model size
    print(f"Quantized model size: {get_model_size(model_quantized)}")
    return model_quantized

quantized_model = quantize_model(model)

100%|██████████| 1/1 [00:41<00:00, 41.22s/it]

tensor([ 65, 795, 230, 809, 520,  60, 334, 415, 674, 332, 109, 286, 370, 757,
        595, 147, 327,  23, 478, 517, 334, 208, 948, 727,  23, 846, 270, 166,
         55, 538, 324, 573, 360, 981, 586, 887,  26, 398, 777,  74, 431, 756,
        129, 198, 256, 725, 565, 166, 717, 467,  92,  29, 844, 591, 359, 468,
        154, 994, 872, 588, 735, 197, 107,  46, 842, 390, 101, 887, 870, 911,
          4, 149,  21, 476,  80, 424, 159, 275, 175, 461, 970, 160, 788,  58,
        479, 498, 368,  28, 487,  50, 270, 383, 366, 780, 373, 705, 330, 142,
        949, 349])
Accuracy of the network on the 100 test images: 81 %


Quantized model size: 51919308
